#NPC Chatbot App with ChatGPT, LangChain and Streamlit

#Install Dependencies

In [ ]:
# Install Dependencies
!pip install langchain==0.1.12
!pip install langchain-openai==0.0.8
!pip install langchain-community==0.0.29
!pip install streamlit==1.32.2
!pip install chromadb==0.4.24
!pip install pyngrok==7.1.5
!pip install nltk

#Load OpenAI API Credentials

In [ ]:
# Load OpenAI API Credentials
from getpass import getpass


OPENAI_KEY = getpass('Enter Open AI API Key: ')


#Set Environment Variable

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = OPENAI_KEY

#Write App Code Header

In [ ]:
%%writefile app.py
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_community.chat_message_histories import StreamlitChatMessageHistory
from langchain.callbacks.base import BaseCallbackHandler
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores.chroma import Chroma
from langchain.chains import LLMChain
from operator import itemgetter

import streamlit as st
import tempfile
import os
import nltk
from nltk.tokenize import sent_tokenize

nltk.download('punkt')

# Customize initial app landing page
st.set_page_config(page_title="NPC Simulation Chatbot", page_icon="🎭")
st.title("NPC Simulation Chatbot 🎭")
st.sidebar.header("NPC Character Document Upload")

@st.cache_resource(ttl="1h")
def configure_npc_agent(uploaded_file):
    if uploaded_file is not None:
        text_content = uploaded_file.read().decode("utf-8")

        # --- Normalize line endings to standard Unix-style (\n) ---
        text_content = text_content.replace('\\r\\n', '\\n').replace('\\r', '\\n')

        # --- SECTION PARSING: Line-based parsing ---
        lines = text_content.strip().split('\n') # Split into lines
        sections_dict = {}
        current_section_name = None
        current_section_content = []

        for line in lines:
            line = line.strip()
            if line.startswith("---") and line.endswith("---"):
                if current_section_name:
                    sections_dict[current_section_name] = "\n".join(current_section_content).strip()
                    current_section_content = [] # Reset content for new section
                current_section_name = line.strip('-').strip() # Extract section name
            elif current_section_name:
                current_section_content.append(line) # Append line to current section

        if current_section_name and current_section_content: # Capture last section
            sections_dict[current_section_name] = "\n".join(current_section_content).strip()


        character_description = sections_dict.get("Character Description", "")
        backstory_world_info = sections_dict.get("Backstory and World Info", "")
        quests_text = sections_dict.get("Quests", "")

        quests = [q.strip() for q in quests_text.strip().split("- ") if q.strip()]

        return character_description, backstory_world_info, quests

    return None, None, None


# Manages live updates to a Streamlit app's display by appending new text tokens
class StreamHandler(BaseCallbackHandler):
    def __init__(self, container, initial_text=""):
        self.container = container
        self.text = initial_text

    def on_llm_new_token(self, token: str, **kwargs) -> None:
        self.text += token
        self.container.markdown(self.text)

# UI element to upload text document
uploaded_file = st.sidebar.file_uploader(
    label="Upload NPC Character Text Document", type=["txt"]
)

if uploaded_file is None:
    st.info("Please upload your NPC Character Text Document to continue.")
    st.stop()

# Configure NPC agent
character_description, backstory_world_info, original_quests = configure_npc_agent(uploaded_file) # Renamed original_quests

if character_description is None or backstory_world_info is None or original_quests is None:
    st.error("Error loading character document. Please check the format.")
    st.stop()

# Initialize quests in session state so it persists across interactions, and make a copy
if "quests" not in st.session_state:
    st.session_state.quests = list(original_quests) # Create a copy to avoid modifying original list each run
quests = st.session_state.quests # Use the session state version

# Split backstory and world info into chunks for RAG
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_overlap=200,
    chunk_size=2000,
    length_function=len,
)
doc_chunks = text_splitter.create_documents([backstory_world_info])

# Create embeddings and vectorstore for backstory/world info
embeddings_model = OpenAIEmbeddings()
vectordb = Chroma.from_documents(doc_chunks, embeddings_model)
retriever = vectordb.as_retriever()


# Load LLM
chatgpt = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.7, streaming=True)

# System prompt for NPC character
system_prompt_template = SystemMessagePromptTemplate.from_template(character_description)

# RAG Prompt for retrieving NPC/World info - IMPROVED for less hallucination and mood
rag_prompt_template = ChatPromptTemplate.from_messages([
    system_prompt_template,
    HumanMessagePromptTemplate.from_template("""Use the following context to answer the user question about yourself or the world.
                                              Context: {context}
                                              Question: {question}
                                              Answer ONLY based on the provided context. Do not invent or fantasize information outside of the context.
                                              If you cannot answer from the context, just say you do not know.
                                              Answer in character as described in the system prompt.
                                              Current NPC Mood: {npc_mood}""")
])


# General conversation prompt (no RAG) - IMPROVED for quest awareness and mood
conversation_prompt_template = ChatPromptTemplate.from_messages([
    system_prompt_template,
    HumanMessagePromptTemplate.from_template("""{question}
                                              Consider your available quests. If the user's question is about a problem, trouble, or something you are concerned about,
                                              or if the user is offering help, and if you have a relevant quest that could address it, subtly hint at or offer the quest in your response.
                                              Even if not directly asked about quests, if the conversation naturally leads to a quest, you can bring it up.
                                              However, do not invent problems or force quests into the conversation if they are not relevant.
                                              Just answer the question in character and weave in quest hints naturally when appropriate.
                                              Current NPC Mood: {npc_mood}
                                              Respond in a way that reflects your current mood.
                                              Mood states are: Happy, Neutral, Slightly Annoyed, Annoyed, Grumpy, Angry.
                                              If Happy, be very cheerful and helpful.
                                              If Neutral, be polite and normal.
                                              If Slightly Annoyed, be a bit terse, less enthusiastic.
                                              If Annoyed, be clearly irritated, give short answers.
                                              If Grumpy, be rude and unhelpful, refuse requests if possible.
                                              If Angry, be very aggressive and refuse to interact much.""")
])


# Quest related prompts - Mood aware refusal
quest_suggestion_prompt_template = ChatPromptTemplate.from_messages([
    system_prompt_template,
    HumanMessagePromptTemplate.from_template("""Consider your current mood when responding to the user's quest request.
                                              Current NPC Mood: {npc_mood}

                                              If your mood is Grumpy, respond rudely and refuse to offer a quest unless the user apologizes.  For example: "You rude... before you apologize I won't have a quest for you."
                                              If your mood is Angry, respond aggressively and refuse to offer a quest or interact further. For example: "Get away from me! I'm in no mood for your games. No quests for you!

                                              Respond in character as described in the system prompt.
                                              """)
])
quest_suggestion_chain = quest_suggestion_prompt_template | chatgpt

# New prompt for when NO quests are available, regardless of mood (can be more neutral)
no_quests_available_prompt_template = ChatPromptTemplate.from_messages([
    system_prompt_template,
    HumanMessagePromptTemplate.from_template("""I'm sorry, but I don't have any quests for you at the moment. Perhaps later.
                                              Respond in character as described in the system prompt.
                                              Current NPC Mood: {npc_mood}""") # Still include mood for consistent character
])
no_quests_available_chain = no_quests_available_prompt_template | chatgpt


def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

rag_chain = (
    {
        "context": itemgetter("question") | retriever | format_docs,
        "question": itemgetter("question"),
        "npc_mood": itemgetter("npc_mood")
    }
    | rag_prompt_template
    | chatgpt
)
conversation_chain = (
    {
        "question": itemgetter("question"),
        "npc_mood": itemgetter("npc_mood")
    }
    | conversation_prompt_template
    | chatgpt
)
quest_suggestion_chain_with_mood = (
    {
        "npc_mood": itemgetter("npc_mood")
    }
    | quest_suggestion_prompt_template
    | chatgpt
)
no_quests_available_chain_with_mood = ( # Chain for "no quests" response
    {
        "npc_mood": itemgetter("npc_mood")
    }
    | no_quests_available_prompt_template
    | chatgpt
)


# --- Question Type Classifier Chain for NPC Chatbot ---
npc_question_type_prompt_template = """
Determine the type of question being asked to the NPC. Choose from the following categories: quest_request, info_request, or general_conversation.

Question: {question}

Respond with 'quest_request' if the question is clearly asking for a quest, or about quests in general, OR if the user is offering to help (e.g., "give me a quest", "offer a quest", "do you have quests?", "what quests are available?", "i want a quest", "tell me about quests", "any tasks for me?", "is there anything I can help you with?", "can I help you?", "I want to help", "is there anything you need help with?", "can i be of assistance?").
Respond with 'info_request' if the question is asking for information about the NPC or their world, requiring context from their background (e.g., "tell me about yourself", "who are you?", "where are we?", "what is this place?", "tell me about the world").
Respond with 'general_conversation' if the question is just a general greeting, statement, or question that doesn't clearly fall into 'quest_request' or 'info_request' (e.g., "hello", "how are you?", "what do you do?", "nice weather today", "what's troubling you?", "is anything wrong?").  Include questions about the NPC's well-being or current state here.

Just answer 'quest_request', 'info_request', or 'general_conversation'.
"""
npc_question_type_prompt = ChatPromptTemplate.from_template(npc_question_type_prompt_template)
npc_question_type_chain = npc_question_type_prompt | chatgpt
# --- End Question Type Classifier Chain ---

# --- User Sentiment Classifier Chain ---
user_sentiment_prompt_template = """
Analyze the user's message and classify their sentiment towards you (the NPC).
Choose one of the following sentiments: 'rude', 'neutral', 'friendly', 'apology'.

Message: {user_message}

Respond with just the sentiment category: 'rude', 'neutral', 'friendly', or 'apology'.
'rude' - if the user is being insulting, offensive, or disrespectful.
'neutral' - if the user is making a question, statement, or greeting without strong emotion.
'friendly' - if the user is being polite, kind, helpful, or positive.
'apology' - if the user is explicitly apologizing for prior behavior.
"""
user_sentiment_prompt = ChatPromptTemplate.from_template(user_sentiment_prompt_template)
user_sentiment_chain = user_sentiment_prompt | chatgpt
# --- End User Sentiment Classifier Chain ---


# Conversation Handling
streamlit_msg_history = StreamlitChatMessageHistory(key="npc_chat_messages")

if "npc_mood" not in st.session_state:
    st.session_state.npc_mood = "Neutral" # Default mood is now Neutral
mood_scale = ["Happy", "Neutral", "Slightly Annoyed", "Annoyed", "Grumpy", "Angry"] # Define mood scale
mood_index = {mood: i for i, mood in enumerate(mood_scale)} # Mood to index mapping
max_negative_mood = "Angry" # Highest negative mood state


if len(streamlit_msg_history.messages) == 0:
    streamlit_msg_history.add_ai_message("Greetings. How may I assist you?") # Neutral greeting

for msg in streamlit_msg_history.messages:
    st.chat_message(msg.type).write(msg.content)

if user_prompt := st.chat_input():
    st.chat_message("human").write(user_prompt)

    # --- Classify User Sentiment ---
    sentiment_response = user_sentiment_chain.invoke({"user_message": user_prompt})
    user_sentiment = sentiment_response.content.strip()
    st.write(f"--- **DEBUGGING: User Sentiment:** --- {user_sentiment}") # Sentiment debug


    # --- Adjust NPC Mood based on User Sentiment ---
    current_mood = st.session_state.npc_mood
    current_mood_index = mood_index[current_mood]
    st.write(f"--- **DEBUGGING: NPC Mood BEFORE:** --- {current_mood}") # Mood debug

    if user_sentiment == "rude":
        new_mood_index = min(current_mood_index + 2, mood_index[max_negative_mood]) # Move down mood scale, max at Angry
    elif user_sentiment == "apology" or user_sentiment == "friendly":
        new_mood_index = max(current_mood_index - 1, mood_index["Neutral"]) # Move up mood scale, min at Neutral
    else: # "neutral" sentiment
        new_mood_index = current_mood_index # Stay the same

    st.session_state.npc_mood = mood_scale[new_mood_index] # Update mood
    st.write(f"--- **DEBUGGING: NPC Mood AFTER:** --- {st.session_state.npc_mood}") # Mood debug


    # --- Classify question type ---
    question_type_response = npc_question_type_chain.invoke({"question": user_prompt})
    question_type = question_type_response.content.strip()
    st.write(f"--- **DEBUGGING: Question Type:** --- {question_type}") # Question type debug


    if question_type == "quest_request":
        if st.session_state.npc_mood in ["Grumpy", "Angry"]: # Refuse quest if grumpy or angry
            with st.chat_message("ai"):
                stream_handler = StreamHandler(st.empty())
                config = {"callbacks": [stream_handler]}
                # Use quest_suggestion_chain to get a mood-appropriate refusal message
                response = quest_suggestion_chain_with_mood.invoke({"npc_mood": st.session_state.npc_mood}, config)
            streamlit_msg_history.add_user_message(user_prompt)
            streamlit_msg_history.add_ai_message(response.content)

        elif quests:
            if quests: # Double check quests are not empty just before accessing (redundant, but safer)
                # Changed from pop(0) to [0] to NOT remove the quest
                quest_to_offer = st.session_state.quests[0] # Get the FIRST quest but do NOT remove it from the list

                # --- Direct Quest Output - Bypassing LLM for Quest Text ---
                with st.chat_message("ai"):
                    quest_intro = "Alright, here's a task for ya then:" # Slightly more neutral intro
                    quest_message = f"{quest_intro}\n\n**{quest_to_offer}**"
                    st.markdown(quest_message)

                streamlit_msg_history.add_user_message(user_prompt)
                streamlit_msg_history.add_ai_message(quest_message)
            else: # Should not reach here normally if outer 'if quests:' is correct, but just in case
                with st.chat_message("ai"):
                    stream_handler = StreamHandler(st.empty())
                    config = {"callbacks": [stream_handler]}
                    response = no_quests_available_chain_with_mood.invoke({"npc_mood": st.session_state.npc_mood}, config) # Use "no quests" chain
                streamlit_msg_history.add_user_message(user_prompt)
                streamlit_msg_history.add_ai_message(response.content)


        else: # No quests available (quests list is empty), even if mood is good
             with st.chat_message("ai"):
                stream_handler = StreamHandler(st.empty())
                config = {"callbacks": [stream_handler]}
                response = no_quests_available_chain_with_mood.invoke({"npc_mood": st.session_state.npc_mood}, config) # Use "no quests" chain
             streamlit_msg_history.add_user_message(user_prompt)
             streamlit_msg_history.add_ai_message(response.content)


    elif question_type == "info_request":
        # RAG for NPC/World info
        with st.chat_message("ai"):
            stream_handler = StreamHandler(st.empty())
            config = {"callbacks": [stream_handler]}
            response = rag_chain.invoke({"question": user_prompt, "npc_mood": st.session_state.npc_mood}, config)
        streamlit_msg_history.add_user_message(user_prompt)
        streamlit_msg_history.add_ai_message(response.content)

    elif question_type == "general_conversation":
        # General conversation
        with st.chat_message("ai"):
            stream_handler = StreamHandler(st.empty())
            config = {"callbacks": [stream_handler]}
            response = conversation_chain.invoke({"question": user_prompt, "npc_mood": st.session_state.npc_mood}, config)
        streamlit_msg_history.add_user_message(user_prompt)
        streamlit_msg_history.add_ai_message(response.content)

    else: # Fallback
        with st.chat_message("ai"):
            stream_handler = StreamHandler(st.empty())
            config = {"callbacks": [stream_handler]}
            response = conversation_chain.invoke({"question": user_prompt, "npc_mood": st.session_state.npc_mood}, config)
        streamlit_msg_history.add_user_message(user_prompt)
        streamlit_msg_history.add_ai_message(response.content)

#Starting the Streamlit App

In [ ]:
# Starting the Streamlit App
!streamlit run app.py --server.port=8989 &>/./logs.txt &

#Setting Up ngrok Tunnel

In [ ]:
# Setting Up ngrok Tunnel
from getpass import getpass

ngrok_auth_token = getpass('Enter ngrok API Key: ')

In [ ]:
from pyngrok import ngrok
import yaml

# Terminate open tunnels if exist
ngrok.kill()

# Authenticate ngrok with the token read from the file
!ngrok config add-authtoken {ngrok_auth_token}

# Open an HTTPS tunnel on port XXXX which you get from your `logs.txt` file
ngrok_tunnel = ngrok.connect(8989)
print("Streamlit App:", ngrok_tunnel.public_url)